In [2]:
import tensorflow as tf
import pandas as pd
# import seaborn as sns

In [3]:
data = pd.read_excel("../UNT-Thermography-24/fused_silica_data/fused_silicia_exact_complete.xlsx")

exact_df = pd.DataFrame(data)

exact_df.head()

,layer 1,layer 2,layer 3,layer 4,layer 5,layer 6,layer 7,layer 8,layer 9,layer 10,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
0,312.355087,301.649641,321.270850,324.903956,329.245296,337.694194,333.776774,333.750946,331.605971,333.828275,...,2.378753e-12,2.416740e-12,2.452565e-12,2.482976e-12,2.505710e-12,2.514345e-12,2.497372e-12,2.443515e-12,2.360022e-12,2.283099e-12
1,308.045992,314.390077,319.612686,324.021871,328.286597,324.268581,318.045216,310.205743,316.395558,315.206667,...,4.690686e-12,4.758857e-12,4.814785e-12,4.855500e-12,4.879557e-12,4.876002e-12,4.823191e-12,4.700047e-12,4.521287e-12,4.356649e-12
2,275.470643,334.023614,330.292063,333.180051,317.824092,306.451798,309.091647,312.060959,320.208051,336.027107,...,2.891349e-12,2.935824e-12,2.976514e-12,3.009965e-12,3.033863e-12,3.040662e-12,3.016554e-12,2.948046e-12,2.844024e-12,2.748188e-12
3,319.154096,314.324236,322.541381,324.154848,333.263850,344.199198,338.490892,334.185374,327.065180,303.761845,...,1.335647e-12,1.361978e-12,1.387258e-12,1.409588e-12,1.427626e-12,1.437648e-12,1.432962e-12,1.406923e-12,1.363502e-12,1.323517e-12
4,368.613557,330.588867,323.002818,322.824169,336.505619,328.080745,325.609686,335.941694,328.415271,330.880143,...,2.861867e-12,2.906931e-12,2.947792e-12,2.981218e-12,3.005087e-12,3.012002e-12,2.988298e-12,2.920602e-12,2.817710e-12,2.722916e-12


In [4]:
data = pd.read_excel("../UNT-Thermography-24/fused_silica_data/fused_silica_snr35_complete.xlsx")

exact_df = pd.DataFrame(data)

exact_df.head()

,layer 1,layer 2,layer 3,layer 4,layer 5,layer 6,layer 7,layer 8,layer 9,layer 10,...,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008,0.000008
0,340.700455,311.405820,330.352308,317.445833,329.463317,336.135380,335.637771,335.311523,349.226485,353.055896,...,3.677670e-12,3.775585e-12,3.769325e-12,3.893884e-12,3.924094e-12,3.918214e-12,3.856920e-12,3.733407e-12,3.640021e-12,3.538380e-12
1,354.001709,316.633317,306.208243,307.242573,304.143102,301.428489,297.332664,297.258058,297.894341,307.547960,...,4.800437e-12,4.878549e-12,4.949850e-12,4.972488e-12,5.001958e-12,4.867123e-12,4.975104e-12,4.770089e-12,4.643758e-12,4.427513e-12
2,282.686476,314.627887,321.703020,328.976737,324.288819,311.672306,299.393223,310.694178,314.497358,332.603920,...,3.241054e-12,3.376236e-12,3.388060e-12,3.388936e-12,3.492327e-12,3.420195e-12,3.437225e-12,3.311146e-12,3.102263e-12,3.089759e-12
3,346.263096,326.978496,335.768963,323.939796,335.604382,336.571570,320.278565,317.790996,328.896079,315.549712,...,3.609418e-12,3.598367e-12,3.705184e-12,3.711047e-12,3.677849e-12,3.717044e-12,3.627688e-12,3.613795e-12,3.401600e-12,3.293201e-12
4,342.024601,324.789014,329.356554,318.193110,318.495015,321.828174,312.840042,306.608068,314.026350,306.537996,...,2.503851e-12,2.570619e-12,2.627113e-12,2.647810e-12,2.653769e-12,2.622466e-12,2.643523e-12,2.610887e-12,2.482391e-12,2.389012e-12
